# Pyspark tutorial 🎯

**Ghi chú: tutorial này được thực hiện trên google colab nên không bị lỗi khi chạy code. Nếu chạy trên local machine chắc chắn sẽ lỗi. Mặc dù đã mò mẫm cả tuần nhưng vẫn không có giải pháp nào cho vấn đề này nên mình chấp nhận thực hiện nó trên google colab. 🤷‍♂️**

## Installation ⬇️

Để làm việc với spark, chúng ta cần cài đặt các thành phần sau:

- 📌 Java (OpenJDK 8 hoặc 11):
  
  <details><summary>Xài ubuntu hoặc các nền tảng như Google Colab thì chạy lệnh này:</summary>
      👉
      <code>!apt-get install openjdk-8-jdk-headless -qq > /dev/null</code>
    </details><br>

  <details>
      <summary>Nếu xài window thì vào đây tải cho lẹ:</summary>
      👉 <a href="https://www.oracle.com/java/technologies/javase/javase8-archive-downloads.html" target="_blank"> www.oracle.com </a>
    </details>

- 📌 Apache Spark

  <details><summary>Nếu xài Ubuntu hoặc Google Colab</summary><br>
    👉 <code>wget https://downloads.apache.org/spark/spark-3.5.4/spark-3.5.4-bin-hadoop3.tgz</code>

    👉 <code>tar -xvzf spark-3.5.4-bin-hadoop3.tgz</code>

    👉 <code>sudo mv spark-3.5.4-bin-hadoop3 /opt/spark</code>
    
    🔧 Thiết lập biến môi trường như sau:

    Với ubuntu:
    ```
    echo "export SPARK_HOME=/opt/spark" >> ~/.bashrc
    echo "export PATH=$SPARK_HOME/bin:$PATH" >> ~/.bashrc
    source ~/.bashrc

    ```
    Với google colab:

    ```
    import os
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
    os.environ["SPARK_HOME"] = "/content/spark-3.5.4-bin-hadoop3"

    ```
    </details><br>

  <details><summary>Nếu xài window thì:</summary><br>
    👉 Tải thủ công tại <a href="https://spark.apache.org/downloads.html" target="_blank"> spark.apache.org </a>

    👉 Rồi giải nén thủ công, có thể đặt ở đâu đó cũng được. Ví dụ: `C:\spark\spark-3.5.4-bin-hadoop3`,

    👉 Thiết lập biến môi trường thủ công, nhớ thêm tên môi trường (chẳng hạn `$SPARK_HOME`)
    
    🚀 Kham khảo chi tiết tại: <a href="https://www.datacamp.com/tutorialpyspark-tutorial-getting-started-with-pyspark" target="_blank">pyspark-tutorial</a>

    </details>
  

- 📌 pyspark

In [ ]:
!pip install pyspark

- 📌 findspark - tùy chọn, nếu chạy trên Google Colab

In [ ]:
!pip install findspark

## SparkSession 🔥

Apache Spark là một hệ thống phân tán, chạy trên nhiều node. Để tương tác với spark, chúng ta cần một thứ gọi là "điểm khởi đầu" (Entry point). Trong trường hợp này chính là SparkSession. SparkSession giống như cửa chính của một tòa nhà, giúp bạn truy cập vào các thành phần bên trong.

In [ ]:
# khởi tạo SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Datacamp Pyspark Tutorial") \
    .config("spark.memory.offHeap.enabled","true") \
    .config("spark.memory.offHeap.size","10g").getOrCreate()


In [ ]:
# Dừng SparkSession khi cần thiết
# spark.stop()

## SparkContext 🔥

🔹SparkContext là lớp cốt lõi trong Apache Spark, giúp ứng dụng của bạn kết nối với Spark Cluster và quản lý các tác vụ tính toán.

🔹Trước Spark 2.x, SparkContext là entry point chính.

🔹Từ Spark 2.x trở đi, SparkSession thay thế SparkContext, nhưng SparkContext vẫn tồn tại bên trong SparkSession.

In [ ]:
from pyspark import SparkContext

# Khởi tạo sparkContext
# sc = SparkContext("local", "MyApp")
# print(sc)

🔹Lưu ý: sparkContext chỉ khởi tạo một lần

🔹Chạy lần 2 bị báo lỗi `Cannot run multiple SparkContexts at once; existing SparkContext`

🔹Có thể khắc phục bằng 2 cách:
    🔸Dừng sparkContext,
    🔸Lấy sparkContext hiện có

In [ ]:
# kiểm tra có SparkContext nào đang chạy không
SparkContext._active_spark_context

<SparkContext master=local[*] appName=Datacamp Pyspark Tutorial>

In [ ]:
# Dừng SparkContext khi cần thiết
# sc.stop()

# Sau đó tạo mới
# sc = SparkContext("local", "MyApp")
# print(sc)

# Lưu ý: Không gọi sc.stop() nếu biến sc chưa tồn tại. Lúc đó phải chạy lệnh bên dưới

In [ ]:
# Lấy SparkContext hiện có từ SparkSession
sc = spark.sparkContext
# print(sc)

## RDD (Resilient Distributed Dataset) 🔥

RDD (Resilient Distributed Dataset) là cấu trúc dữ liệu cốt lõi của Apache Spark, đại diện cho một tập dữ liệu phân tán có thể chịu lỗi và xử lý song song. Đây là cấp độ trừu tượng thấp nhất trong Spark, cung cấp nền tảng cho các API cấp cao hơn như DataFrame và Dataset.

In [ ]:
words = [
    'Scalar',
    'java',
    'hadoop',
    'spark',
    'akka',
    'spark and hadoop',
    'pyspark',
    'pyspark and spark'
]

print(words)
print(type(words))

['Scalar', 'java', 'hadoop', 'spark', 'akka', 'spark and hadoop', 'pyspark', 'pyspark and spark']
<class 'list'>


In [ ]:
rdd_words = sc.parallelize(words)
print(type(rdd_words))

<class 'pyspark.rdd.RDD'>


In [ ]:
print(rdd_words.count())

8


**Thao tác với RDD 🪄**

Các phép biến đổi trên RDD tạo ra RDD mới mà không thay đổi RDD ban đầu.
Một số phép biến đổi phổ biến:

- map(f): Áp dụng một hàm f lên từng phần tử.
- filter(f): Lọc các phần tử thỏa mãn điều kiện f.
- flatMap(f): Giống map(), nhưng mở rộng kết quả thành nhiều phần tử.
- groupByKey(): Nhóm các phần tử có cùng khóa (key-value RDD).
- reduceByKey(f): Kết hợp các giá trị có cùng khóa bằng một hàm f.